# Filter and downsample the h5ad files
### Downsample count matrices and meta files are saved as CSVs for BayesPrism

In [2]:
import scanpy as sc
import pandas as pd
import anndata as ad
import numpy as np
import scipy
from scipy.sparse import csr_matrix

# Macula

In [2]:
# Load anndata object
macula_adata = sc.read_h5ad('macula_counts_unfilter_071024.h5ad')
mac_meta = macula_adata.obs

# Get cell and gene name lists
cells_list = macula_adata.obs_names.tolist()
genes_list = macula_adata.var_names.tolist()

# Create df to filter
mac_df = macula_adata.to_df()

# cell type counts BEFORE any filtering
mac_meta['class'].value_counts()

class
RGC        75131
AC         29855
Rods       15550
BC-OFF      2716
BC-ON       2673
MG          1635
mlCones      970
HC-H1        890
HC-H2        179
Astro        113
sCones        92
Name: count, dtype: int64

In [3]:
#Filter cells to have >= 200 genes
sc.pp.filter_cells(macula_adata, min_genes=200) 

#Filter genes to be expressed in >= 50 cells
sc.pp.filter_genes(macula_adata, min_cells=50) 

macula_adata.obs['class'].value_counts()

class
RGC        75124
AC         29833
Rods       15498
BC-OFF      2709
BC-ON       2671
MG          1627
mlCones      968
HC-H1        890
HC-H2        179
Astro        113
sCones        92
Name: count, dtype: int64

In [5]:
macula_adata.shape

(129704, 27450)

In [4]:
# Remove >5% mito
macula_adata = macula_adata[macula_adata.obs['percent.mt'] < 5]
macula_adata.obs['class'].value_counts()

class
RGC        75119
AC         29805
Rods       15451
BC-OFF      2707
BC-ON       2666
MG          1627
mlCones      964
HC-H1        889
HC-H2        179
Astro        113
sCones        92
Name: count, dtype: int64

In [5]:
# Keep top 99.7%
mac_meta = macula_adata.obs

cell_types = mac_meta['class'].unique().tolist()

mac_final = pd.DataFrame()

for ct in cell_types:
    df = mac_meta[mac_meta['class'] == ct]
    max_genes = df['nFeature_RNA'].max()
    threshhold = max_genes * 0.997
    df_filt = df[df['nFeature_RNA'] < threshhold]

    #print(df_filt)

    mac_final = pd.concat([mac_final, df_filt], ignore_index=False)

mac_final['class'].value_counts()

class
RGC        75118
AC         29804
Rods       15450
BC-OFF      2706
BC-ON       2665
MG          1626
mlCones      963
HC-H1        888
HC-H2        178
Astro        112
sCones        91
Name: count, dtype: int64

In [6]:
# Select cells to keep and genes to keep [cells, genes]
cells = mac_final.index.to_list()
genes = macula_adata.var.index.to_list() 

# Filter df
mac_filtered = mac_df.loc[cells, genes]

mac_filtered.shape

(129601, 27450)

In [7]:
# Load gencode v26
gencode = pd.read_csv("gencode.v26.broad.category.txt", sep="\t", header=None)
duplicated = pd.read_csv('gencode_duplicated.csv')
gencode_filt = gencode[gencode[8].isin(['lincRNA', 'protein_coding'])]
gencode_filt = gencode_filt[~gencode_filt[0].isin(['chrY', 'chrX', 'chrM'])]
gencode_filt = gencode_filt[~gencode_filt[4].isin(duplicated.x.to_list())]

# Filter based off of gencode
df_genes = mac_filtered.columns.tolist()
merged = gencode_filt[gencode_filt[4].isin(df_genes)] # using gene names
to_keep = merged[4]
mac_gene_filt = mac_filtered[to_keep]

# Make anndata of the filtered data
cells_list = mac_final.index.tolist()
genes_list = mac_gene_filt.columns.tolist()

mac_counts = csr_matrix(mac_gene_filt)
anndata = ad.AnnData(mac_counts)

anndata.obs_names = cells_list
anndata.var_names = genes_list
anndata.obs = mac_final
anndata.write_h5ad('filter_macula_071124.h5ad')

In [10]:
mac_gene_filt.shape

(129601, 15097)

In [8]:
anndata.obs['class'].value_counts()

class
RGC        75118
AC         29804
Rods       15450
BC-OFF      2706
BC-ON       2665
MG          1626
mlCones      963
HC-H1        888
HC-H2        178
Astro        112
sCones        91
Name: count, dtype: int64

In [11]:
# Subset 
rgc = anndata[anndata.obs['class'].isin(['RGC'])]
ac = anndata[anndata.obs['class'].isin(['AC'])]
rods = anndata[anndata.obs['class'].isin(['Rods'])]
rest = anndata[~anndata.obs['class'].isin(['RGC', 'AC', 'Rods'])]

# Subsample to <5000
sc.pp.subsample(rgc, fraction=0.06)
sc.pp.subsample(ac, fraction=0.16)
sc.pp.subsample(rods, fraction=0.32)

# Merge them again
merged = ad.concat([rgc, ac, rods, rest])
merged.obs['class'].value_counts()

# Get updated meta and df
meta = merged.obs
cells = meta.index.tolist()
genes = to_keep
df = mac_filtered.loc[cells, genes]

# Save
meta.to_csv('meta_downsample5k_macula_071124.csv')
df.to_csv('downsample5k_macula_071124.csv')

In [10]:
meta['class'].value_counts()

class
Rods       4944
AC         4768
RGC        4507
BC-OFF     2706
BC-ON      2665
MG         1626
mlCones     963
HC-H1       888
HC-H2       178
Astro       112
sCones       91
Name: count, dtype: int64

# PERIPHERAL

In [14]:
# Load anndata object
periph_adata = sc.read_h5ad('periph_counts_unfilter_071124.h5ad')
per_meta = periph_adata.obs

# Get cell and gene name lists
cells_list = periph_adata.obs_names.tolist()
genes_list = periph_adata.var_names.tolist()

# Create df to filter
per_df = periph_adata.to_df()

# cell type counts BEFORE any filtering
periph_adata.obs['class'].value_counts()

class
Rods       21935
AC         20936
RGC         4901
BC-ON       2260
BC-OFF      1477
MG          1438
mlCones      966
HC-H1        482
Astro        350
HC-H2        203
sCones        56
Name: count, dtype: int64

In [15]:
#Filter cells to have >= 200 genes
sc.pp.filter_cells(periph_adata, min_genes=200)

#Filter genes to be expressed in >= 50 cells
sc.pp.filter_genes(periph_adata, min_cells=50) 

periph_adata.obs['class'].value_counts()

class
Rods       21923
AC         20924
RGC         4898
BC-ON       2260
BC-OFF      1474
MG          1437
mlCones      960
HC-H1        482
Astro        350
HC-H2        203
sCones        56
Name: count, dtype: int64

In [114]:
periph_adata.shape

(54967, 22835)

In [16]:
# Remove >5% mito
periph_adata = periph_adata[periph_adata.obs['percent.mt'] < 5]
periph_adata.obs['class'].value_counts()

class
Rods       21919
AC         20897
RGC         4897
BC-ON       2260
BC-OFF      1473
MG          1433
mlCones      960
HC-H1        482
Astro        350
HC-H2        203
sCones        56
Name: count, dtype: int64

In [17]:
# Keep top 99.7%
per_meta = periph_adata.obs

cell_types = per_meta['class'].unique().tolist()

per_final = pd.DataFrame()

for ct in cell_types:
    df = per_meta[per_meta['class'] == ct]
    max_genes = df['nFeature_RNA'].max()
    threshhold = max_genes * 0.997
    df_filt = df[df['nFeature_RNA'] < threshhold]

    #print(df_filt)

    per_final = pd.concat([per_final, df_filt], ignore_index=False)

per_final['class'].value_counts()

class
Rods       21918
AC         20896
RGC         4896
BC-ON       2259
BC-OFF      1472
MG          1432
mlCones      959
HC-H1        481
Astro        349
HC-H2        202
sCones        55
Name: count, dtype: int64

In [18]:
# Select cells to keep and genes to keep [cells, genes]
cells = per_final.index.to_list()
genes = periph_adata.var.index.to_list() 

# Filter df
per_filtered = per_df.loc[cells, genes]

per_filtered.shape

(54919, 22835)

In [19]:
# Load gencode v26
gencode = pd.read_csv("gencode.v26.broad.category.txt", sep="\t", header=None)
duplicated = pd.read_csv('gencode_duplicated.csv')
gencode_filt = gencode[gencode[8].isin(['lincRNA', 'protein_coding'])]
gencode_filt = gencode_filt[~gencode_filt[0].isin(['chrY', 'chrX', 'chrM'])]
gencode_filt = gencode_filt[~gencode_filt[4].isin(duplicated.x.to_list())]

# Filter based off of gencode
df_genes = per_filtered.columns.tolist()
merged = gencode_filt[gencode_filt[4].isin(df_genes)] # using gene names
to_keep = merged[4]
per_gene_filt = per_filtered[to_keep]

In [121]:
per_gene_filt.shape

(54919, 14056)

In [22]:
# Make anndata of the filtered data
cells_list = per_final.index.tolist()
genes_list = per_gene_filt.columns.tolist()

per_counts = csr_matrix(per_gene_filt)
anndata = ad.AnnData(per_counts)

anndata.obs_names = cells_list
anndata.var_names = genes_list
anndata.obs = per_final

# Subset rods and AC
rods_ac = anndata[anndata.obs['class'].isin(['Rods', 'AC'])]
rest = anndata[~anndata.obs['class'].isin(['Rods', 'AC'])]

# Subsample to <5000
sc.pp.subsample(rods_ac, fraction=0.2)

# Merge them again
merged = ad.concat([rods_ac, rest])
merged.obs['class'].value_counts()

# Get updated meta and df
meta = merged.obs
cells = meta.index.tolist()
genes = to_keep
df = per_filtered.loc[cells, genes]

# Save
meta.to_csv('meta_downsample5k_periph_071124.csv')
df.to_csv('downsample5k_periph_071124.csv')

In [21]:
meta['class'].value_counts()

class
RGC        4896
Rods       4378
AC         4184
BC-ON      2259
BC-OFF     1472
MG         1432
mlCones     959
HC-H1       481
Astro       349
HC-H2       202
sCones       55
Name: count, dtype: int64